# DATA 612 Project 4 | Accuracy and Beyond 

### Choose a different dataset to work with from your previous projects

### 1. Compare the accuracy of at least two recommender system algorithms against your offline data. Choose two recommenders, then pick a metric. you can look at RMSE and MEA, but pick something recommender specific like novelty, diversity, serendipity. compare between the three metrics, be critical, address why that recommender is best. offline data is data not used for training

### 2. Implement support for at least one business or user experience goal such as increased serendipity, novelty, or diversity. look beyond the model but its use. Example: if using a content base filtering and user keep watching action movies, you will reach a point when your exhsaus t your action movie libaries and user has nothing to new to watch. what can you do to avoid this, and find a way to explore if te user also likes other gemres  without just suggestin  a random different genre, just percieved as noise

### 3. Compare and report on any change in accuracy before and after you’ve made the change in #2.

### 4. As part of your textual conclusion, discuss one or more additional experiments that could be performed and/or metrics that could be evaluated only if online evaluation was possible. Also, briefly propose how you would design a reasonable online evaluation environment.

Compare the accuracy of at least two recommender algorithms using offline data and metrics
such as RMSE, precision, or recall to evaluate the performance of each model

In [ ]:
Implement and evaluate business or user experience goals (e.g., increased serendipity, novelty,
or diversity) within a recommender system, and assess the impact of these factors on user
satisfaction and system performance.

Analyze and report on changes in accuracy after implementing enhancements related to business goals, comparing the performance of the recommender system before and after modifications

Design and propose experiments for online evaluation, outlining how online metrics could be
used to assess the real-world effectiveness of recommender systems, including user engagement,
conversion rates, or retention.

Explore the limitations of offline evaluation metrics and explain the importance of online
evaluation for assessing a recommender system’s real-world performance and relevance.

Propose a framework for online evaluation, including how to set up an A/B testing environment, handle user interactions, and gather real-time data to further optimize and validate
recommender algorithms.

By completing this module, students will gain practical experience in fine-tuning recommender
systems to align with business goals and user preferences, and learn how to move beyond
traditional accuracy metrics to evaluate the success of their systems in a more holistic, realworld contex